# eBird API - Species & Time Period Downloader
Download bird observations by **region code** OR **geographic location** for any time period.

**New Feature**: Download specific species across all European countries!

## 1. Setup and Imports

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from typing import List, Optional
import json

## 2. Configuration

In [ ]:
# Your eBird API key
API_KEY = "tmb363lktsir"

# ========== CHOOSE ONE MODE ==========
# Mode 1: Single Region/Country
USE_SINGLE_REGION = False
COUNTRY_CODE = "RO"  # Romania
REGION_CODE = None    # Optional: use specific region like "RO-CV" for Covasna

# Mode 2: Multiple Countries (e.g., all of Europe)
USE_MULTIPLE_COUNTRIES = True
# European country codes (eBird uses ISO 3166-1 alpha-2 codes)
EUROPEAN_COUNTRIES = [
    'AL', 'AD', 'AT', 'BY', 'BE', 'BA', 'BG', 'HR', 'CY', 'CZ',
    'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT',
    'XK', 'LV', 'LI', 'LT', 'LU', 'MT', 'MD', 'MC', 'ME', 'NL',
    'MK', 'NO', 'PL', 'PT', 'RO', 'RU', 'SM', 'RS', 'SK', 'SI',
    'ES', 'SE', 'CH', 'TR', 'UA', 'GB', 'VA'
]

# Or use a custom list of countries
CUSTOM_COUNTRIES = []  # e.g., ['RO', 'BG', 'HU'] for Romania, Bulgaria, Hungary

# Mode 3: By Geographic Coordinates (limited to recent data)
USE_LOCATION = False
LATITUDE = 45.8438205
LONGITUDE = 26.183044
RADIUS_KM = 50

# Date range
START_DATE = "2024-01-01"  # Format: YYYY-MM-DD
END_DATE = "2024-01-31"    # Format: YYYY-MM-DD

# Species filter - ADD YOUR TARGET SPECIES HERE
TARGET_SPECIES = ['dalpel1']  # Example: Dalmatian Pelican

# API parameters
MAX_RESULTS_PER_REQUEST = 500
INCLUDE_PROVISIONAL = True
HOTSPOT_ONLY = False

# Rate limiting (important when querying multiple countries)
DELAY_BETWEEN_REQUESTS = 0.5  # seconds between date requests
DELAY_BETWEEN_COUNTRIES = 2.0  # seconds between countries

## 3. Helper Functions

In [ ]:
def generate_date_range(start_date: str, end_date: str) -> List[str]:
    """Generate list of dates in eBird API format (YYYY/M/D)."""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    date_list = []
    current_date = start
    
    while current_date <= end:
        date_str = f"{current_date.year}/{current_date.month}/{current_date.day}"
        date_list.append(date_str)
        current_date += timedelta(days=1)
    
    return date_list


def fetch_observations_by_region(region: str, date: str, api_key: str,
                                 max_results: int = 500, 
                                 hotspot: bool = False,
                                 include_provisional: bool = True) -> List[dict]:
    """Fetch bird observations for a specific date and region."""
    url = f"https://api.ebird.org/v2/data/obs/{region}/historic/{date}"
    
    params = {
        "detail": "full",
        "hotspot": str(hotspot).lower(),
        "maxResults": max_results,
        "includeProvisional": str(include_provisional).lower(),
        "sppLocale": "en"
    }
    
    headers = {"X-eBirdApiToken": api_key}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {date}: {e}")
        return []


def filter_by_species(observations: List[dict], species_codes: List[str]) -> List[dict]:
    """Filter observations by species codes."""
    if not species_codes:
        return observations
    return [obs for obs in observations if obs.get('speciesCode') in species_codes]


def collect_observations_by_region(region: str, date_range: List[str], api_key: str,
                                   species_filter: Optional[List[str]] = None,
                                   max_results: int = 500,
                                   hotspot: bool = False,
                                   include_provisional: bool = True,
                                   delay: float = 0.5) -> List[dict]:
    """Collect observations across multiple dates."""
    all_observations = []
    
    for i, date in enumerate(date_range, 1):
        observations = fetch_observations_by_region(
            region=region, date=date, api_key=api_key,
            max_results=max_results, hotspot=hotspot,
            include_provisional=include_provisional
        )
        
        if species_filter:
            observations = filter_by_species(observations, species_filter)
        
        all_observations.extend(observations)
        
        if i < len(date_range):
            time.sleep(delay)
    
    return all_observations


def collect_observations_multi_country(countries: List[str], 
                                       date_range: List[str],
                                       api_key: str,
                                       species_filter: Optional[List[str]] = None,
                                       max_results: int = 500,
                                       hotspot: bool = False,
                                       include_provisional: bool = True,
                                       delay_requests: float = 0.5,
                                       delay_countries: float = 2.0) -> List[dict]:
    """Collect observations across multiple countries."""
    all_observations = []
    country_stats = []
    
    print(f"\n{'='*70}")
    print(f"MULTI-COUNTRY DOWNLOAD")
    print(f"{'='*70}")
    print(f"Countries: {len(countries)}")
    print(f"Date range: {len(date_range)} days")
    print(f"Species filter: {species_filter if species_filter else 'All species'}")
    print(f"{'='*70}\n")
    
    for country_idx, country in enumerate(countries, 1):
        print(f"\n[{country_idx}/{len(countries)}] Processing {country}...")
        print("-" * 50)
        
        country_observations = collect_observations_by_region(
            region=country, date_range=date_range, api_key=api_key,
            species_filter=species_filter, max_results=max_results,
            hotspot=hotspot, include_provisional=include_provisional,
            delay=delay_requests
        )
        
        all_observations.extend(country_observations)
        country_stats.append({
            'country': country,
            'observations': len(country_observations)
        })
        
        print(f"✓ {country}: {len(country_observations)} observations")
        
        if country_idx < len(countries):
            time.sleep(delay_countries)
    
    print(f"\n{'='*70}")
    print(f"SUMMARY")
    print(f"{'='*70}")
    print(f"Total observations: {len(all_observations)}")
    print(f"\nObservations by country:")
    stats_df = pd.DataFrame(country_stats).sort_values('observations', ascending=False)
    print(stats_df.to_string(index=False))
    
    return all_observations

## 4. Fetch Data

In [ ]:
# Generate date range
date_range = generate_date_range(START_DATE, END_DATE)

if USE_MULTIPLE_COUNTRIES:
    # Multi-country mode
    countries = CUSTOM_COUNTRIES if CUSTOM_COUNTRIES else EUROPEAN_COUNTRIES
    
    print(f"Using {'CUSTOM' if CUSTOM_COUNTRIES else 'ALL EUROPEAN'} countries ({len(countries)} total)")
    
    observations = collect_observations_multi_country(
        countries=countries,
        date_range=date_range,
        api_key=API_KEY,
        species_filter=TARGET_SPECIES,
        max_results=MAX_RESULTS_PER_REQUEST,
        hotspot=HOTSPOT_ONLY,
        include_provisional=INCLUDE_PROVISIONAL,
        delay_requests=DELAY_BETWEEN_REQUESTS,
        delay_countries=DELAY_BETWEEN_COUNTRIES
    )

elif USE_SINGLE_REGION:
    # Single region mode
    region = REGION_CODE if REGION_CODE else COUNTRY_CODE
    print(f"Fetching data for region: {region}")
    print(f"Date range: {len(date_range)} days\n")
    
    observations = collect_observations_by_region(
        region=region,
        date_range=date_range,
        api_key=API_KEY,
        species_filter=TARGET_SPECIES,
        max_results=MAX_RESULTS_PER_REQUEST,
        hotspot=HOTSPOT_ONLY,
        include_provisional=INCLUDE_PROVISIONAL,
        delay=DELAY_BETWEEN_REQUESTS
    )
    
    print(f"\nTotal observations: {len(observations)}")

else:
    print("Please set USE_SINGLE_REGION=True or USE_MULTIPLE_COUNTRIES=True")
    observations = []

## 5. Convert to DataFrame and Explore

In [ ]:
df = pd.DataFrame(observations)

if len(df) > 0:
    df['obsDt'] = pd.to_datetime(df['obsDt'])
    df = df.sort_values('obsDt')
    
    print(f"\nDataFrame shape: {df.shape}")
    print(f"Date range: {df['obsDt'].min()} to {df['obsDt'].max()}")
    print(f"Countries: {df['countryCode'].nunique()}")
    
    display(df.head(10))
else:
    print("No observations found!")

## 6. Summary Statistics

In [ ]:
if len(df) > 0:
    print("=" * 60)
    print("SUMMARY STATISTICS")
    print("=" * 60)
    
    # Species summary
    print("\n📊 Species Summary:")
    species_counts = df.groupby(['speciesCode', 'comName']).size().reset_index(name='count')
    species_counts = species_counts.sort_values('count', ascending=False)
    print(species_counts.to_string(index=False))
    
    # Country summary (if multiple countries)
    if df['countryCode'].nunique() > 1:
        print("\n🌍 Observations by Country:")
        country_counts = df.groupby(['countryCode', 'countryName']).size().reset_index(name='count')
        country_counts = country_counts.sort_values('count', ascending=False)
        print(country_counts.to_string(index=False))
    
    # Observations by date
    print("\n📅 Observations by Date:")
    daily_counts = df.groupby(df['obsDt'].dt.date).size()
    print(daily_counts.to_string())
    
    # Top locations
    print("\n📍 Top Locations:")
    location_counts = df.groupby(['locName', 'countryCode']).size().reset_index(name='count')
    location_counts = location_counts.sort_values('count', ascending=False).head(10)
    print(location_counts.to_string(index=False))
    
    print(f"\n👥 Unique Observers: {df['userDisplayName'].nunique()}")
    print(f"📌 Unique Locations: {df['locName'].nunique()}")
    
    if 'howMany' in df.columns:
        total_birds = df['howMany'].sum()
        print(f"🐦 Total Individual Birds Counted: {total_birds}")

## 7. Export Data

In [ ]:
if len(df) > 0:
    # Define filename
    if USE_MULTIPLE_COUNTRIES:
        if CUSTOM_COUNTRIES:
            location_str = f"countries_{'_'.join(CUSTOM_COUNTRIES[:3])}"
        else:
            location_str = "europe_all_countries"
    else:
        region = REGION_CODE if REGION_CODE else COUNTRY_CODE
        location_str = f"region_{region}"
    
    species_str = "_".join(TARGET_SPECIES) if TARGET_SPECIES else "all_species"
    filename = f"ebird_{location_str}_{species_str}_{START_DATE}_to_{END_DATE}.csv"
    
    # Export to CSV
    df.to_csv(filename, index=False)
    print(f"✅ Data exported to: {filename}")
    
    # Export to JSON
    json_filename = filename.replace('.csv', '.json')
    df.to_json(json_filename, orient='records', date_format='iso', indent=2)
    print(f"✅ Data exported to: {json_filename}")
    
    # Export country summary if applicable
    if USE_MULTIPLE_COUNTRIES and df['countryCode'].nunique() > 1:
        summary_filename = filename.replace('.csv', '_country_summary.csv')
        country_summary = df.groupby(['countryCode', 'countryName']).agg({
            'obsId': 'count',
            'speciesCode': 'nunique',
            'locName': 'nunique'
        }).reset_index()
        country_summary.columns = ['countryCode', 'countryName', 'observations', 'unique_species', 'unique_locations']
        country_summary = country_summary.sort_values('observations', ascending=False)
        country_summary.to_csv(summary_filename, index=False)
        print(f"✅ Country summary exported to: {summary_filename}")
else:
    print("⚠️ No data to export")

## 8. Visualizations (Optional)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

if len(df) > 0:
    # Country distribution (if multiple countries)
    if USE_MULTIPLE_COUNTRIES and df['countryCode'].nunique() > 1:
        fig, ax = plt.subplots(figsize=(12, 8))
        country_counts = df['countryName'].value_counts().head(20)
        country_counts.plot(kind='barh', ax=ax, color='forestgreen')
        ax.set_xlabel('Number of Observations', fontsize=12)
        ax.set_ylabel('Country', fontsize=12)
        ax.set_title('Top 20 Countries by Observations', fontsize=14, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Time series
    fig, ax = plt.subplots(figsize=(12, 6))
    daily_obs = df.groupby(df['obsDt'].dt.date).size()
    daily_obs.plot(kind='bar', ax=ax, color='steelblue')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Number of Observations', fontsize=12)
    ax.set_title(f'Daily Observations: {START_DATE} to {END_DATE}', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 9. Helper: Find Species Codes

In [ ]:
def search_species(search_term: str, region: str = "RO") -> pd.DataFrame:
    """Search for species codes by common name."""
    url = f"https://api.ebird.org/v2/data/obs/{region}/recent"
    params = {"maxResults": 500}
    headers = {"X-eBirdApiToken": API_KEY}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    
    species_df = pd.DataFrame(data)[['speciesCode', 'comName', 'sciName']].drop_duplicates()
    mask = species_df['comName'].str.contains(search_term, case=False, na=False)
    
    return species_df[mask].reset_index(drop=True)

# Example: search_species("pelican")